In [1]:
import numpy as np
from sqlalchemy import create_engine
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.svm import SVC
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

In [2]:
filepath = '../Resources/BMI_Datasets/overweight.csv'
medical_df = pd.read_csv(filepath)
medical_df.head(10)

,ID#,Gender,Age,Hypertension,Heart_Disease,Ever_Married,Work_Type,Residence_Type,Avg_Glucose_Lvl,BMI,Smoker,Stroke
0,6,Male,81,0,0,Yes,Private,Urban,186.21,29.0,Former,1
1,7,Male,74,1,1,Yes,Private,Rural,70.09,27.4,Never,1
2,11,Female,81,1,0,Yes,Private,Rural,80.43,29.7,Never,1
3,13,Female,54,0,0,Yes,Private,Urban,104.51,27.3,Current,1
4,15,Female,79,0,1,Yes,Private,Urban,214.09,28.2,Never,1
5,18,Male,75,1,0,Yes,Private,Urban,221.29,25.8,Current,1
6,23,Female,79,0,0,Yes,Self-employed,Urban,228.70,26.6,Never,1
7,25,Male,71,0,0,Yes,Private,Urban,102.87,27.2,Former,1
8,27,Female,65,0,0,Yes,Private,Rural,100.98,28.2,Former,1
9,29,Male,69,0,1,Yes,Self-employed,Urban,195.23,28.3,Current,1


In [3]:
# Drop ID# column
medical_df.drop(columns=['ID#'], inplace=True)
medical_df.head()

,Gender,Age,Hypertension,Heart_Disease,Ever_Married,Work_Type,Residence_Type,Avg_Glucose_Lvl,BMI,Smoker,Stroke
0,Male,81,0,0,Yes,Private,Urban,186.21,29.0,Former,1
1,Male,74,1,1,Yes,Private,Rural,70.09,27.4,Never,1
2,Female,81,1,0,Yes,Private,Rural,80.43,29.7,Never,1
3,Female,54,0,0,Yes,Private,Urban,104.51,27.3,Current,1
4,Female,79,0,1,Yes,Private,Urban,214.09,28.2,Never,1


In [4]:
medical_df['Stroke'].value_counts()

0    1334
1      75
Name: Stroke, dtype: int64

In [6]:
# Get a list of categorical columns
categorical_columns = medical_df.dtypes[medical_df.dtypes=='object'].index.tolist()
categorical_columns

['Gender', 'Ever_Married', 'Work_Type', 'Residence_Type', 'Smoker']

In [7]:
# Create OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit & transform OneHotEncoder using categorical columns
encode_df = pd.DataFrame(enc.fit_transform(medical_df[categorical_columns]))

# Add column names
encode_df.columns = enc.get_feature_names(categorical_columns)
print(encode_df.shape)
encode_df.head(10)

(1409, 15)


C:\Users\remem\anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Gender_Female,Gender_Male,Ever_Married_No,Ever_Married_Yes,Work_Type_Govt_job,Work_Type_Never_worked,Work_Type_Private,Work_Type_Self-employed,Work_Type_children,Residence_Type_Rural,Residence_Type_Urban,Smoker_Current,Smoker_Former,Smoker_Never,Smoker_Unknown
0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
4,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
5,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
6,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
7,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
8,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
9,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


In [8]:
# Get encode_df info
encode_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1409 entries, 0 to 1408
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Gender_Female            1409 non-null   float64
 1   Gender_Male              1409 non-null   float64
 2   Ever_Married_No          1409 non-null   float64
 3   Ever_Married_Yes         1409 non-null   float64
 4   Work_Type_Govt_job       1409 non-null   float64
 5   Work_Type_Never_worked   1409 non-null   float64
 6   Work_Type_Private        1409 non-null   float64
 7   Work_Type_Self-employed  1409 non-null   float64
 8   Work_Type_children       1409 non-null   float64
 9   Residence_Type_Rural     1409 non-null   float64
 10  Residence_Type_Urban     1409 non-null   float64
 11  Smoker_Current           1409 non-null   float64
 12  Smoker_Former            1409 non-null   float64
 13  Smoker_Never             1409 non-null   float64
 14  Smoker_Unknown          

In [9]:
# Drop redundant columns
encode_df.drop(columns=['Gender_Female', 'Ever_Married_No', 'Residence_Type_Rural'], inplace=True)
encode_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1409 entries, 0 to 1408
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Gender_Male              1409 non-null   float64
 1   Ever_Married_Yes         1409 non-null   float64
 2   Work_Type_Govt_job       1409 non-null   float64
 3   Work_Type_Never_worked   1409 non-null   float64
 4   Work_Type_Private        1409 non-null   float64
 5   Work_Type_Self-employed  1409 non-null   float64
 6   Work_Type_children       1409 non-null   float64
 7   Residence_Type_Urban     1409 non-null   float64
 8   Smoker_Current           1409 non-null   float64
 9   Smoker_Former            1409 non-null   float64
 10  Smoker_Never             1409 non-null   float64
 11  Smoker_Unknown           1409 non-null   float64
dtypes: float64(12)
memory usage: 132.2 KB


In [10]:
# Merge encoded df with medical df
medical_df = medical_df.merge(encode_df, left_index=True, right_index=True).drop(categorical_columns, axis=1)
print(medical_df.shape)
medical_df.head(10)

(1409, 18)


,Age,Hypertension,Heart_Disease,Avg_Glucose_Lvl,BMI,Stroke,Gender_Male,Ever_Married_Yes,Work_Type_Govt_job,Work_Type_Never_worked,Work_Type_Private,Work_Type_Self-employed,Work_Type_children,Residence_Type_Urban,Smoker_Current,Smoker_Former,Smoker_Never,Smoker_Unknown
0,81,0,0,186.21,29.0,1,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,74,1,1,70.09,27.4,1,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,81,1,0,80.43,29.7,1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,54,0,0,104.51,27.3,1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
4,79,0,1,214.09,28.2,1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
5,75,1,0,221.29,25.8,1,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
6,79,0,0,228.70,26.6,1,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
7,71,0,0,102.87,27.2,1,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
8,65,0,0,100.98,28.2,1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9,69,0,1,195.23,28.3,1,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0


In [11]:
medical_df.isnull().sum()

Age                        0
Hypertension               0
Heart_Disease              0
Avg_Glucose_Lvl            0
BMI                        0
Stroke                     0
Gender_Male                0
Ever_Married_Yes           0
Work_Type_Govt_job         0
Work_Type_Never_worked     0
Work_Type_Private          0
Work_Type_Self-employed    0
Work_Type_children         0
Residence_Type_Urban       0
Smoker_Current             0
Smoker_Former              0
Smoker_Never               0
Smoker_Unknown             0
dtype: int64

In [12]:
# Create feature & target datasets
X = medical_df.drop(columns=['Stroke'])
y = medical_df['Stroke']
print(X.shape)
print(y.shape)

(1409, 17)
(1409,)


In [13]:
# Create empty arrays to store ML results
recalls = []
accuracies = []
precisions = []

# This will take a while. Feel free to adjust range to sample code. Feel free to review exported spreadsheets in github.
for i in range(1000):
    
    # Create training + testing data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
    
    # Oversample positive stroke cases in training set
    ros = RandomOverSampler()
    X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)
    
    # Scale feature data
    scaler = StandardScaler()
    scaler.fit(X_train_resampled)
    X_train_scaled = scaler.transform(X_train_resampled)
    X_test_scaled = scaler.transform(X_test)
    
    # Create SVC model
    svc_model = SVC(kernel='linear')
    svc_model = svc_model.fit(X_train_scaled, y_train_resampled)
    
    # Evaluate SVC model
    y_pred = svc_model.predict(X_test_scaled)
    
    # Add recall results to list
    recalls.append(recall_score(y_test, y_pred))
    
    # Add precision results to list
    precisions.append(precision_score(y_test, y_pred))
    
    # Add accuracy results to list
    accuracies.append(accuracy_score(y_test, y_pred))
    
    print("Current progress:", i)

Current progress: 0
Current progress: 1
Current progress: 2
Current progress: 3
Current progress: 4
Current progress: 5
Current progress: 6
Current progress: 7
Current progress: 8
Current progress: 9
Current progress: 10
Current progress: 11
Current progress: 12
Current progress: 13
Current progress: 14
Current progress: 15
Current progress: 16
Current progress: 17
Current progress: 18
Current progress: 19
Current progress: 20
Current progress: 21
Current progress: 22
Current progress: 23
Current progress: 24
Current progress: 25
Current progress: 26
Current progress: 27
Current progress: 28
Current progress: 29
Current progress: 30
Current progress: 31
Current progress: 32
Current progress: 33
Current progress: 34
Current progress: 35
Current progress: 36
Current progress: 37
Current progress: 38
Current progress: 39
Current progress: 40
Current progress: 41
Current progress: 42
Current progress: 43
Current progress: 44
Current progress: 45
Current progress: 46
Current progress: 47
Cu

Current progress: 378
Current progress: 379
Current progress: 380
Current progress: 381
Current progress: 382
Current progress: 383
Current progress: 384
Current progress: 385
Current progress: 386
Current progress: 387
Current progress: 388
Current progress: 389
Current progress: 390
Current progress: 391
Current progress: 392
Current progress: 393
Current progress: 394
Current progress: 395
Current progress: 396
Current progress: 397
Current progress: 398
Current progress: 399
Current progress: 400
Current progress: 401
Current progress: 402
Current progress: 403
Current progress: 404
Current progress: 405
Current progress: 406
Current progress: 407
Current progress: 408
Current progress: 409
Current progress: 410
Current progress: 411
Current progress: 412
Current progress: 413
Current progress: 414
Current progress: 415
Current progress: 416
Current progress: 417
Current progress: 418
Current progress: 419
Current progress: 420
Current progress: 421
Current progress: 422
Current pr

Current progress: 751
Current progress: 752
Current progress: 753
Current progress: 754
Current progress: 755
Current progress: 756
Current progress: 757
Current progress: 758
Current progress: 759
Current progress: 760
Current progress: 761
Current progress: 762
Current progress: 763
Current progress: 764
Current progress: 765
Current progress: 766
Current progress: 767
Current progress: 768
Current progress: 769
Current progress: 770
Current progress: 771
Current progress: 772
Current progress: 773
Current progress: 774
Current progress: 775
Current progress: 776
Current progress: 777
Current progress: 778
Current progress: 779
Current progress: 780
Current progress: 781
Current progress: 782
Current progress: 783
Current progress: 784
Current progress: 785
Current progress: 786
Current progress: 787
Current progress: 788
Current progress: 789
Current progress: 790
Current progress: 791
Current progress: 792
Current progress: 793
Current progress: 794
Current progress: 795
Current pr

In [14]:
recalls

[0.7333333333333333,
 0.7333333333333333,
 0.8666666666666667,
 0.8666666666666667,
 0.8666666666666667,
 0.5333333333333333,
 0.5333333333333333,
 0.6666666666666666,
 0.6,
 0.5333333333333333,
 0.8666666666666667,
 0.6666666666666666,
 0.7333333333333333,
 0.8,
 0.6666666666666666,
 0.6666666666666666,
 0.8666666666666667,
 0.8666666666666667,
 0.6666666666666666,
 0.8666666666666667,
 0.9333333333333333,
 0.9333333333333333,
 0.8666666666666667,
 0.6666666666666666,
 0.8666666666666667,
 0.6666666666666666,
 0.7333333333333333,
 0.5333333333333333,
 0.6666666666666666,
 0.6,
 0.8,
 0.7333333333333333,
 0.9333333333333333,
 0.6666666666666666,
 0.8,
 0.8666666666666667,
 0.8666666666666667,
 0.7333333333333333,
 0.7333333333333333,
 0.8666666666666667,
 0.7333333333333333,
 0.8666666666666667,
 0.6666666666666666,
 0.7333333333333333,
 0.7333333333333333,
 0.4,
 0.5333333333333333,
 0.7333333333333333,
 0.8666666666666667,
 0.8,
 0.8,
 0.8,
 0.6666666666666666,
 0.8666666666666667,
 

In [15]:
recalls_df = pd.DataFrame(recalls, columns=['Recall'])
print(recalls_df.shape)
recalls_df.head(10)

(1000, 1)


,Recall
0,0.733333
1,0.733333
2,0.866667
3,0.866667
4,0.866667
5,0.533333
6,0.533333
7,0.666667
8,0.600000
9,0.533333


In [16]:
accuracies

[0.7056737588652482,
 0.7092198581560284,
 0.6879432624113475,
 0.6595744680851063,
 0.7056737588652482,
 0.7056737588652482,
 0.6773049645390071,
 0.6808510638297872,
 0.6666666666666666,
 0.7056737588652482,
 0.7092198581560284,
 0.6914893617021277,
 0.7269503546099291,
 0.6879432624113475,
 0.6950354609929078,
 0.6453900709219859,
 0.6808510638297872,
 0.6666666666666666,
 0.7163120567375887,
 0.7021276595744681,
 0.6950354609929078,
 0.6666666666666666,
 0.6914893617021277,
 0.6914893617021277,
 0.7446808510638298,
 0.6808510638297872,
 0.6560283687943262,
 0.6773049645390071,
 0.6560283687943262,
 0.6524822695035462,
 0.6879432624113475,
 0.6666666666666666,
 0.6595744680851063,
 0.6595744680851063,
 0.6808510638297872,
 0.7092198581560284,
 0.6950354609929078,
 0.7092198581560284,
 0.7198581560283688,
 0.7127659574468085,
 0.7163120567375887,
 0.6312056737588653,
 0.7163120567375887,
 0.7340425531914894,
 0.6985815602836879,
 0.7163120567375887,
 0.6950354609929078,
 0.6914893617

In [17]:
accuracies_df = pd.DataFrame(accuracies, columns=['Accuracy'])
print(accuracies_df.shape)
accuracies_df.head(10)

(1000, 1)


,Accuracy
0,0.705674
1,0.709220
2,0.687943
3,0.659574
4,0.705674
5,0.705674
6,0.677305
7,0.680851
8,0.666667
9,0.705674


In [18]:
precisions

[0.12222222222222222,
 0.12359550561797752,
 0.13131313131313133,
 0.12149532710280374,
 0.13829787234042554,
 0.09523809523809523,
 0.08695652173913043,
 0.10526315789473684,
 0.09278350515463918,
 0.09523809523809523,
 0.13978494623655913,
 0.10869565217391304,
 0.13095238095238096,
 0.12371134020618557,
 0.10989010989010989,
 0.09523809523809523,
 0.12871287128712872,
 0.12380952380952381,
 0.11764705882352941,
 0.1368421052631579,
 0.1414141414141414,
 0.1308411214953271,
 0.1326530612244898,
 0.10869565217391304,
 0.1566265060240964,
 0.10526315789473684,
 0.10576923076923077,
 0.08695652173913043,
 0.09803921568627451,
 0.0891089108910891,
 0.12371134020618557,
 0.10891089108910891,
 0.12844036697247707,
 0.09900990099009901,
 0.12121212121212122,
 0.13978494623655913,
 0.13402061855670103,
 0.12359550561797752,
 0.12790697674418605,
 0.14130434782608695,
 0.12643678160919541,
 0.11304347826086956,
 0.11764705882352941,
 0.13414634146341464,
 0.11956521739130435,
 0.0779220779220

In [19]:
precisions_df = pd.DataFrame(precisions, columns=['Precision'])
print(precisions_df.shape)
precisions_df.head(10)

(1000, 1)


,Precision
0,0.122222
1,0.123596
2,0.131313
3,0.121495
4,0.138298
5,0.095238
6,0.086957
7,0.105263
8,0.092784
9,0.095238


In [20]:
results_df = accuracies_df.merge(precisions_df, left_index=True, right_index=True)
print(results_df.shape)
results_df.head(10)

(1000, 2)


,Accuracy,Precision
0,0.705674,0.122222
1,0.709220,0.123596
2,0.687943,0.131313
3,0.659574,0.121495
4,0.705674,0.138298
5,0.705674,0.095238
6,0.677305,0.086957
7,0.680851,0.105263
8,0.666667,0.092784
9,0.705674,0.095238


In [21]:
results_df = results_df.merge(recalls_df, left_index=True, right_index=True)
print(results_df.shape)
results_df.head(10)

(1000, 3)


,Accuracy,Precision,Recall
0,0.705674,0.122222,0.733333
1,0.709220,0.123596,0.733333
2,0.687943,0.131313,0.866667
3,0.659574,0.121495,0.866667
4,0.705674,0.138298,0.866667
5,0.705674,0.095238,0.533333
6,0.677305,0.086957,0.533333
7,0.680851,0.105263,0.666667
8,0.666667,0.092784,0.600000
9,0.705674,0.095238,0.533333


In [22]:
print("Recall mean: " + str(results_df['Recall'].mean()))
print("Recall mode: " + str(results_df['Recall'].mode()))
print("Recall median: " + str(results_df['Recall'].median()))
print("Recall max: " + str(results_df['Recall'].max()))
print("Recall min: " + str(results_df['Recall'].min()))
print("Recall standard deviation: " + str(results_df['Recall'].std()))

Recall mean: 0.7309333333333345
Recall mode: 0    0.733333
dtype: float64
Recall median: 0.7333333333333333
Recall max: 1.0
Recall min: 0.3333333333333333
Recall standard deviation: 0.11410781711723646


In [23]:
print("Accuracy mean: " + str(results_df['Accuracy'].mean()))
print("Accuracy mode: " + str(results_df['Accuracy'].mode()))
print("Accuracy median: " + str(results_df['Accuracy'].median()))
print("Accuracy max: " + str(results_df['Accuracy'].max()))
print("Accuracy min: " + str(results_df['Accuracy'].min()))
print("Accuracy standard deviation: " + str(results_df['Accuracy'].std()))

Accuracy mean: 0.6912907801418438
Accuracy mode: 0    0.684397
dtype: float64
Accuracy median: 0.6914893617021277
Accuracy max: 0.7801418439716312
Accuracy min: 0.6028368794326241
Accuracy standard deviation: 0.030525914964491675


In [24]:
print("Precision mean: " + str(results_df['Precision'].mean()))
print("Precision mode: " + str(results_df['Precision'].mode()))
print("Precision median: " + str(results_df['Precision'].median()))
print("Precision max: " + str(results_df['Precision'].max()))
print("Precision min: " + str(results_df['Precision'].min()))
print("Precision standard deviation: " + str(results_df['Precision'].std()))

Precision mean: 0.11696660970595853
Precision mode: 0    0.111111
1    0.125000
dtype: float64
Precision median: 0.11764705882352941
Precision max: 0.16470588235294117
Precision min: 0.061855670103092786
Precision standard deviation: 0.016290214806055334


In [25]:
results_df.isnull().sum().sum()

0

In [26]:
len(results_df)

1000

In [27]:
filepath = ('ML_result_files/SVC_Overweight_SimpleImputer(mean)_RandomOverSampler.csv')
results_df.to_csv(filepath, index=False)